$$- \Delta u(x) = f(x) \hspace{0.2cm} \text{on} \hspace{0.2cm} D(\omega_2)$$

In [1]:
from helpers import *

In [2]:
MESH_RESOLUTION_RANDOM_FIELD_CALCULATION = 4

In [ ]:
# Understand constant basis functions
mesh = mshr.generate_mesh(DOMAIN, MESH_RESOLUTION_RANDOM_FIELD_CALCULATION)
V = fe.FunctionSpace(mesh, "DG", 0)
test_function = fe.Function(V)
test_function.vector()[0] = 1
c = fe.plot(test_function, title='Test Function')
plt.colorbar(c)

dof_coordinates = V.tabulate_dof_coordinates().reshape((-1, 2))
print(dof_coordinates[0])
plt.scatter(dof_coordinates[0][0], dof_coordinates[0][1], color='red', label='dof_coordinates[0]')

# Identify the cell containing the DOF
dof_index = 0
cell_index = V.dofmap().cell_dofs(dof_index)[0]
cell = fe.Cell(mesh, cell_index)

# Extract the vertex coordinates of the cell
vertex_indices = cell.entities(0)
vertex_coords = np.array([mesh.coordinates()[vertex] for vertex in vertex_indices])
print("Vertex Coordinates of the Cell:", vertex_coords)

# Plot the vertices of the cell
plt.scatter(vertex_coords[:, 0], vertex_coords[:, 1], color='blue', label='Cell Vertices')

plt.legend()
plt.show()

In [ ]:
randomFieldV, jacobianV = calculate_vector_field_eigenpairs(MESH_RESOLUTION_RANDOM_FIELD_CALCULATION)
# for MESH_RESOLUTION_RANDOM_FIELD_CALCULATION = 8 it takes approx. 3 minutes
# for MESH_RESOLUTION_RANDOM_FIELD_CALCULATION = 6 it takes approx. 47 seconds

In [ ]:
# Plot the eigenvalues
plt.figure()
plt.title('Eigenvalues of the Random Field')
plt.plot(randomFieldV.eigenvalues, 'o')
plt.xlabel('Eigenvalue Index')
plt.ylabel('Eigenvalue')
plt.show()

In [ ]:
xi = np.random.uniform(-np.sqrt(3), np.sqrt(3), randomFieldV.J)
mesh = mshr.generate_mesh(DOMAIN, MESH_RESOLUTION_RANDOM_FIELD_CALCULATION)
# perturbed mesh based on the "original" mesh used for the KL-expansion
perturbed_coordinates = mesh.coordinates().copy()
for index, coordinate in enumerate(mesh.coordinates()):
    perturbed_coordinates[index] = randomFieldV(coordinate, xi)
# Create a new mesh with the perturbed coordinates
perturbed_mesh = fe.Mesh(mesh)
perturbed_mesh.coordinates()[:] = perturbed_coordinates

# Plot the original mesh and the perturbed mesh in one figure with different colors
plt.figure()
fe.plot(mesh, color='blue', linewidth=0.5, label='Original Mesh')
fe.plot(perturbed_mesh, color='red', linewidth=0.5, label='Perturbed Mesh')
plt.legend()
plt.title(f"Original and Perturbed Mesh")
plt.show()

In [ ]:
import time

len_xi = randomFieldV.J
xi = np.random.uniform(-np.sqrt(3), np.sqrt(3), len_xi)

if len_xi > randomFieldV.J:
    raise ValueError("len(xi) must be less than randomFieldV.J")

mesh_resolution_solution = 4
time_start = time.time()
u_sol = solve_poisson_for_given_sample(mesh_resolution_solution, jacobianV, xi, RHS_F)
time_end = time.time()
print(f"Time taken for solving the Poisson problem: {time_end - time_start} seconds")

c = fe.plot(u_sol, title="Poisson solution")
plt.colorbar(c)
plt.show()

In [ ]:
from helpers import calculate_vector_field_eigenpairs, incoherence_fe_edges_visualisation
randomFieldV, jacobianV = calculate_vector_field_eigenpairs(mesh_resolution=4)
incoherence_fe_edges_visualisation(randomFieldV=randomFieldV, mesh_resolution=4)

In [ ]:
from helpers import calculate_vector_field_eigenpairs, inverse_mapping_visualisation
randomFieldV, jacobianV = calculate_vector_field_eigenpairs(mesh_resolution=4)
inverse_mapping_visualisation(mesh_resolution = 4,
                              randomFieldV = randomFieldV,
                              jacobianV = jacobianV)


In [ ]:
# Sample time comparison
from helpers import plot_mesh, measure_time_one_sample 
# resolution 8 takes 10 seconds per sample
# resolution 10 takes 20 seconds per sample

kl_ress = [10, 14]
fem_res = 10
for kl_res in kl_ress:
    randomFieldV, jacobianV = calculate_vector_field_eigenpairs(kl_res)
    plot_mesh(kl_res, randomFieldV)
    measure_time_one_sample(randomFieldV.J, kl_res, fem_res, randomFieldV, jacobianV)

In [ ]:
# Investigation determinant of the Jacobian

det_J_expr = detJExpression(jacobianV, xi, degree=2)
mesh = mshr.generate_mesh(DOMAIN, 5)
V = fe.FunctionSpace(mesh, "P", 1)
det_J_func = fe.project(det_J_expr, V)

c = fe.plot(det_J_func, title="Determinant of the Jacobian")
plt.colorbar(c)
plt.show()

In [ ]:
# Investigation eigenfunctions
from helpers import *

fine_resoluton = 8 #! here use same as in sobol calculation

randomFieldV, jacobianV = calculate_vector_field_eigenpairs(fine_resoluton)
mesh_fine = mshr.generate_mesh(DOMAIN, fine_resoluton)
eval_eigenfunc_k = lambda x, k: np.array([sum([randomFieldV.basis_functions[j].function(x) * randomFieldV.eigenvectors[j, k] for j in range(randomFieldV.N)]), 
                                          sum([randomFieldV.basis_functions[j].function(x) * randomFieldV.eigenvectors[j + randomFieldV.N, k] for j in range(randomFieldV.N)])])

x_coords = mesh_fine.coordinates()[:, 0]
y_coords = mesh_fine.coordinates()[:, 1]
grid_x, grid_y = np.mgrid[-1:1:500j, -1:1:500j]


# First component
fig, axs = plt.subplots(3, 3, figsize=(20, 15))

for k in range(9):
    ax = axs[k // 3, k % 3]
    z_values = []
    for i in range(len(x_coords)):
        z_values.append(eval_eigenfunc_k([x_coords[i], y_coords[i]], k)[0])
    
    grid_z = griddata((x_coords, y_coords), z_values, (grid_x, grid_y), method='linear')
    cp = ax.contourf(grid_x, grid_y, grid_z, levels=100, cmap='viridis')
    cbar = plt.colorbar(cp)
    ax.set_title(f'Basis function {k} to eigenvalue {round(randomFieldV.eigenvalues[k], 5)}')
    ax.set_xlabel(r'$x_1$')
    ax.set_ylabel(r'$x_2$')

plt.tight_layout()
plt.show()


# Second component
fig, axs = plt.subplots(3, 3, figsize=(20, 15))

for k in range(9):
    ax = axs[k // 3, k % 3]
    z_values = []
    for i in range(len(x_coords)):
        z_values.append(eval_eigenfunc_k([x_coords[i], y_coords[i]], k)[1])
    
    grid_z = griddata((x_coords, y_coords), z_values, (grid_x, grid_y), method='linear')
    cp = ax.contourf(grid_x, grid_y, grid_z, levels=100, cmap='viridis')
    cbar = plt.colorbar(cp)
    ax.set_title(f'Eigenfunction {k} to eigenvalue {round(randomFieldV.eigenvalues[k], 5)}')
    ax.set_xlabel(r'$x_1$')
    ax.set_ylabel(r'$x_2$')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt

fine_resoluton = 6

randomFieldV, jacobianV = calculate_vector_field_eigenpairs(fine_resoluton)
mesh_fine = mshr.generate_mesh(DOMAIN, fine_resoluton)
eval_eigenfunc_k = lambda x, k: np.array([sum([randomFieldV.basis_functions[j].function(x) * randomFieldV.eigenvectors[j, k] for j in range(randomFieldV.N)]), 
                                          sum([randomFieldV.basis_functions[j].function(x) * randomFieldV.eigenvectors[j + randomFieldV.N, k] for j in range(randomFieldV.N)])])

x_coords = mesh_fine.coordinates()[:, 0]
y_coords = mesh_fine.coordinates()[:, 1]
grid_x, grid_y = np.mgrid[-1:1:50j, -1:1:50j]  # Adjust the grid resolution as needed

fig, axs = plt.subplots(2, 2, figsize=(15, 15))  # Adjust the figure size as needed

for k in range(4):
    ax = axs[k // 2, k % 2]
    u_values = []
    v_values = []
    for i in range(len(x_coords)):
        basis_function = eval_eigenfunc_k([x_coords[i], y_coords[i]], k)
        u_values.append(basis_function[0])  # x-component
        v_values.append(basis_function[1])  # y-component
    
    # Interpolate the u and v components on the grid
    grid_u = griddata((x_coords, y_coords), u_values, (grid_x, grid_y), method='linear')
    grid_v = griddata((x_coords, y_coords), v_values, (grid_x, grid_y), method='linear')

    # Create the vector plot
    cp = ax.quiver(grid_x, grid_y, grid_u, grid_v, scale=1, scale_units='xy', angles='xy', cmap='viridis')
    eigenvalue_rounded = round(randomFieldV.eigenvalues[k], 5)
    ax.set_title(f'Eigenfunction {k} to eigenvalue {eigenvalue_rounded}')
    ax.set_xlabel(r'$x_1$')
    ax.set_ylabel(r'$x_2$')

plt.tight_layout()
plt.show()

In [ ]:
from helpers import analyse_two_resolutions_from_data_u_hat
import fenics as fe

analyse_two_resolutions_from_data_u_hat(resolution_sparse = 10,
                                  resolution_fine = 14,
                                  P_hat = fe.Point(0.2, 0.2))

In [ ]:
from helpers import calculate_vector_field_eigenpairs, poisson_sobol_calc_indices_from_data, poisson_plot_sobols

mesh_res = 8
size_of_xi = 8
randomFieldV, jacobianV = calculate_vector_field_eigenpairs(mesh_res)

S_single, S_total, mc_sample_size = poisson_sobol_calc_indices_from_data(fem_res=mesh_res, kl_res=mesh_res, size_of_xi=size_of_xi, randomFieldV=randomFieldV, jacobianV=jacobianV)
poisson_plot_sobols(S_single, S_total, mc_sample_size, title=f"Sobol Indices for Poisson Problem Mesh resolution 4")

In [ ]:
from helpers import calculate_vector_field_eigenpairs, poisson_sobol_calc_indices_from_data, poisson_plot_sobols

randomFieldV_4, jacobianV_4 = calculate_vector_field_eigenpairs(4)
S_single_4, S_total_4, mc_sample_size_4 = poisson_sobol_calc_indices_from_data(fem_res=4, kl_res=4, size_of_xi=6, randomFieldV=randomFieldV_4, jacobianV=jacobianV_4)
poisson_plot_sobols(S_single_4, S_total_4, mc_sample_size_4, title=f"Sobol Indices for Poisson Problem Mesh resolution 4")